In [1]:
import requests

In [2]:
url = "https://www.worldometers.info/coronavirus"

In [3]:
req = requests.get(url)

In [4]:
req

<Response [200]>

In [5]:
from lxml.html import fromstring

In [6]:
root = fromstring(req.text)

In [7]:
table = root.xpath(".//table[@id='main_table_countries_today'][1]")[0]

In [8]:
table

<Element table at 0x7efbe83a7b30>

In [9]:
thead = table.xpath(".//thead/tr/th")

In [10]:
def is_visible(attr):
    if 'style' in attr.attrib:
        style = attr.get('style')
        if style.__contains__("display:none") or style.__contains__("display: none"):
            return False
    return True

In [11]:
columns = list()

In [12]:
for th in thead:
    if is_visible(th) is True:
        columns.append(th.text_content().replace('\n', '').replace(u'\xa0', u'').strip())

In [13]:
columns

['Country,Other',
 'TotalCases',
 'NewCases',
 'TotalDeaths',
 'NewDeaths',
 'TotalRecovered',
 'ActiveCases',
 'Serious,Critical',
 'TotCases/1M pop',
 'Deaths/1M pop',
 'TotalTests',
 'Tests/1M pop']

In [14]:
tbody = table.xpath(".//tbody/tr")

In [15]:
tags = list()
values = list()

In [16]:
for tr in tbody:
    if is_visible(tr) is True:
        tag = tr.xpath(".//td/a")
        if len(tag) > 0:
            tags.append(tag[0].get('href').strip().replace('country/', '').replace('/', ''))

        values.append([value.text_content().strip() for value in tr.xpath(".//td") if is_visible(value) is True])

In [17]:
import pandas as pd

In [18]:
data = pd.DataFrame(values, columns=columns)

In [19]:
data.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,World,"3,012,389","+19,127","207,885",+970,"888,073","1,916,431","57,540",386,26.7,,
1,USA,"987,322",+162,"55,415",+2,"118,781","813,126","15,143","2,983",167,"5,470,555","16,527"
2,Spain,"229,422","+2,793","23,521",+331,"120,832","85,069","7,764","4,907",503,"1,268,321","27,127"
3,Italy,"197,675",,"26,644",,"64,928","106,103","2,009","3,269",441,"1,757,659","29,071"
4,France,"162,100",,"22,856",,"44,903","94,341","4,682","2,483",350,"463,662","7,103"


In [20]:
data.to_csv('../data/overview.csv')